卷积运算
<img src="image/conv.jpg"  width="400" >
多个filter卷积示意图
<img src="image/2-5.gif"  width="400" >

### LeNet-5模型
<img src="image/LeNet-5.jpg"  width="400" >
第一层：卷积层

这一层的输入的原始的图像像素，该模型接受的图像为32321，6个5*5卷积核，步长为1，不使用全0填充。所以这层输出的尺寸为32-5+1=28，深度为6。

第二层：池化层

该层的输入为第一层的输出，是一个28286的节点矩阵。本层采用的过滤器大小为22，长和宽的步长均为2，所以本层的输出矩阵大小为1414*6。

第三层：卷积层

本层的输入矩阵大小为14146，16个55卷积核，同样不使用全0填充，步长为1，则本层的输出为1010*16。

第四层：池化层

该层使用22的过滤器，步长为2，故本层的输出矩阵为55*16。

第五层：全连接层

如上文所说，在全连接层之前，需要将5516的矩阵“压扁”为一个向量。本层的输出节点个数为120。

第六层：全连接层

该层输出节点个数为84。

第七层：全连接层

最后一层，输出节点个数为10，样本的标签个数。

## mnist分类

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K  #默认tensorflow后台
K.image_data_format()

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'channels_last'

### 读取数据

In [2]:
local_path='F:/xx/data/mnist/mnist.npz'
(x_train, y_train), (x_test, y_test) = mnist.load_data(path=local_path)

### 处理X和y

In [4]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# 归一化
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
#y转为one-hot
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### LeNet-5模型的搭建  
卷积+最大池化+卷积+最大池化+FC1+FC2+softmax

In [8]:
def LeNet5():
    model = Sequential()
    # 模型第一层要指定input_shape
    model.add(Conv2D(filters=6, kernel_size=(5, 5), padding='valid',activation='tanh',input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=16, kernel_size=(5, 5), padding='valid',activation='tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation='tanh'))
    model.add(Dense(84, activation='tanh'))
    #softmax激活函数用于计算该输入图片属于0-9数字的概率
    model.add(Dense(10, activation='softmax'))
    return model

### 训练模型

In [9]:
model=LeNet5()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=64,
          epochs=20,
          verbose=1,
          validation_split=0.2,
          shuffle=True)


Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 8s 173us/step - loss: 0.2260 - acc: 0.9354 - val_loss: 0.0765 - val_acc: 0.9772
Epoch 2/20
48000/48000 [==============================] - 4s 78us/step - loss: 0.0704 - acc: 0.9792 - val_loss: 0.0618 - val_acc: 0.9807
Epoch 3/20
48000/48000 [==============================] - 4s 75us/step - loss: 0.0493 - acc: 0.9846 - val_loss: 0.0580 - val_acc: 0.9822
Epoch 4/20
48000/48000 [==============================] - 4s 75us/step - loss: 0.0363 - acc: 0.9882 - val_loss: 0.0511 - val_acc: 0.9860
Epoch 5/20
48000/48000 [==============================] - 4s 77us/step - loss: 0.0289 - acc: 0.9912 - val_loss: 0.0518 - val_acc: 0.9854
Epoch 6/20
48000/48000 [==============================] - 4s 78us/step - loss: 0.0221 - acc: 0.9931 - val_loss: 0.0512 - val_acc: 0.9858
Epoch 7/20
48000/48000 [==============================] - 4s 79us/step - loss: 0.0198 - acc: 0.9938 - val_loss: 0.0555 - val_ac

### 评估模型

In [11]:
#返回测试集的损失和准确率
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0503504181388962
Test accuracy: 0.9871


最终在测试集的准确率可以达到98.7%